# Live Object Detection 

In this notebook, you will have the opportunity to check how does your model perform in a real-time situation. 
For that, we will use the OpenCv library.

## Set-Up Workspace

In [ ]:
import os
import os
import tensorflow as tf
import numpy as np
import cv2

In [ ]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet'  # Name of the Network we are going to use 
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8' 
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

In [ ]:
paths = {
    'WORKSPACE_PATH': os.path.join('MyFirstTFOD','Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('MyFirstTFOD','Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('MyFirstTFOD','Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('MyFirstTFOD','Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('MyFirstTFOD','Tensorflow', 'workspace','images'),
    'MODEL_PATH': os.path.join('MyFirstTFOD','Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('MyFirstTFOD','Tensorflow', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('MyFirstTFOD','Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH': os.path.join('MyFirstTFOD','Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'export'), 
    'TFJS_PATH':os.path.join('MyFirstTFOD','Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join('MyFirstTFOD','Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport'), 
    'PROTOC_PATH':os.path.join('MyFirstTFOD','Tensorflow','protoc')
 }

files = {
    'PIPELINE_CONFIG':os.path.join('MyFirstTFOD','Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

## Extract Model *.tar* File (Google Colab only)

If you have trained your model using Google Colab you need to Download the *models.tar.gz* file from your Drive and paste it into the main directory of this repository.
Then just run the code below

In [ ]:
# extract model tar file (#Colab only)
MODEL_FILES = os.path.join('models.tar.gz')
if os.path.exists(MODEL_FILES):
  !tar -zxvf {MODEL_FILES}

## Install TensorFlow Object Detection API (Colab Only) 

If you have trained your model using google colab you probably didn't install the Object Detection API in your local machine.
To proceed with this notebook you need to install it by running the code below.

In [ ]:
if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']} 

In [ ]:
# Install Tensorflow Object Detection 
if os.name=='posix':  
    #!brew install protobuf-compiler # this is for mac os distro
    #!apt-get install protobuf-compiler # this is for linux distro
    !cd MyFirstTFOD/Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install . 
    
if os.name=='nt':
    url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
    wget.download(url)
    !move protoc-3.15.6-win64.zip {paths['PROTOC_PATH']}
    !cd {paths['PROTOC_PATH']} && tar -xf protoc-3.15.6-win64.zip
    os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['PROTOC_PATH'], 'bin'))   
    !cd MyFirstTFOD/Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
    !cd MyFirstTFOD/Tensorflow/models/research/slim && pip install -e .

    Run the verification code bellow

In [ ]:
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
!python {VERIFICATION_SCRIPT}

## Load Trained Model 
Now we are going to import object detection and retrieve the trained model from the last checkpoint.

In [ ]:
import object_detection
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

In [ ]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'], 'ckpt-3')).expect_partial()

@tf.function
def detect_fn(tensor):
    image, shapes = detection_model.preprocess(tensor)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

## Real Time Detections from your Webcam (OpenCv)


    If the code below gives a error regarding the installation of OpenCV, even though you have it installed consider uninstalling and reinstalling the open cv headless
``!pip uninstall opencv-python-headless -y ``

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'])

cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

while cap.isOpened(): 
    ret, frame = cap.read()
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.8,
                agnostic_mode=False)

    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break